# HOUSE PRICE PREDICTION

## Librerias a usar

Importo las librerias que se usaran en la solucion del reto

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error,  make_scorer

RMSLE = make_scorer(mean_squared_log_error, greater_is_better=False, squared=False)

import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

## Extraccion de los Dataframes

In [3]:
df_test = pd.read_csv("houses_test_raw.csv")
df_train = pd.read_csv("house_train_raw.csv")
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


## Transformacion de los datos

Cree funciones para eliminar los features que contengan mayor cantidad de valores perdidos</br>
así como tambien una funcion de preprocesamiento en donde se insertan los datos en crudo y</br>
los devuelve estandarizados y en valores numericos

In [4]:
def percentage_of_missing_values(df_train, df_test):
    # Remove the ID and label columns
    target = 'SalePrice'
    features = pd.concat(
        (df_train.drop(columns=['Id', target]),
         df_test.drop(columns=['Id']))
    )
    top_missing = features.isnull().sum()/len(features)*100
    return top_missing.sort_values(ascending=False)[:4]

df_train.drop(columns=list(percentage_of_missing_values(df_train, df_test).index), inplace=True)
df_test.drop(columns=list(percentage_of_missing_values(df_train, df_test).index), inplace=True)

In [5]:
def preprocess(df_train, df_test):
    # Remove the ID and label columns
    target = 'SalePrice'
    features = pd.concat(
        (df_train.drop(columns=['Id', target]),
         df_test.drop(columns=['Id']))
    )
    # Standardize numerical columns
    numeric_features = features.dtypes[features.dtypes != 'object'].index
    standar_scaler = StandardScaler()
    features[numeric_features] = standar_scaler.fit_transform(features[numeric_features])

    # Replace NAN numerical features by 0
    features[numeric_features] = features[numeric_features].fillna(0)
    
    # Replace discrete features by one-hot encoding.      
    features = pd.get_dummies(features, dummy_na=True)

    # Save preprocessed features
    df_train_new = features[:df_train.shape[0]].copy()
    df_train_new[target] = df_train[target]
    df_test_new = features[df_test.shape[0]:].copy()
    
    return df_train_new, df_test_new

In [6]:
data_train, data_test = preprocess(df_train, df_test)
data_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan,SalePrice
0,0.067331,-0.184481,-0.217879,0.646183,-0.507284,1.046258,0.896833,0.523129,0.580807,-0.29308,...,1,0,0,0,0,0,1,0,0,208500
1,-0.873616,0.458190,-0.072044,-0.063185,2.188279,0.154764,-0.395604,-0.569991,1.177910,-0.29308,...,1,0,0,0,0,0,1,0,0,181500
2,0.067331,-0.055946,0.137197,0.646183,-0.507284,0.980221,0.848965,0.333506,0.097856,-0.29308,...,1,0,0,0,0,0,1,0,0,223500
3,0.302568,-0.398704,-0.078385,0.646183,-0.507284,-1.859351,-0.682812,-0.569991,-0.494856,-0.29308,...,1,0,1,0,0,0,0,0,0,140000
4,0.067331,0.629569,0.518903,1.355551,-0.507284,0.947203,0.753229,1.382009,0.468851,-0.29308,...,1,0,0,0,0,0,1,0,0,250000


Separados los datos en inputs y output

In [7]:
X = data_train.drop(columns=["SalePrice"])
y = data_train["SalePrice"]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

## RANDOM FOREST REGRESSOR

Como modelo para dar solucion a este reto elegí RandomForestRegressor de scikit-learn</br>
ya que dada la naturaleza de nuestros datos me parecio la mejor elección sobre una </br>
regresion lineal u otros métodos

In [9]:
RF = RandomForestRegressor()

In [10]:
RF.fit(X_train, y_train)

RandomForestRegressor()

In [11]:
print('Training score for Random Forest Regressor is',RF.score(X_train,y_train))
print('Testing score for Random Forest Regressor is',RF.score(X_test,y_test))

Training score for Random Forest Regressor is 0.9771156555607842
Testing score for Random Forest Regressor is 0.8829969846745084


In [12]:
RMSLE__ = mean_squared_log_error(y_test,RF.predict(X_test),squared=False)
RMSLE__

0.15389861205463762

## Hyper parametric Tuning -- Random Forest Regressor

In [13]:
random_params = {'n_estimators': [100,200,300,400,500,600],  
               'max_features': [1.0,'sqrt', 'log2', None],  
               'max_depth': [ 10, 15, 20, 25, 60, 70, 80, 90, 100], 
               'min_samples_split':  [2, 5, 10], 
               'min_samples_leaf': [1, 2, 4], 
               'bootstrap': [True, False]}

In [14]:
hyper_tuning = RandomizedSearchCV(
    estimator=RandomForestRegressor(),
    param_distributions=random_params, 
    cv=KFold(n_splits=5,
            shuffle=True,
            random_state=42),
    scoring=RMSLE,
    n_iter=15,
    verbose=5
)

In [15]:
hyper_tuning.fit(X_train,y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END bootstrap=True, max_depth=80, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=400;, score=-0.149 total time=   4.5s
[CV 2/5] END bootstrap=True, max_depth=80, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=400;, score=-0.153 total time=   4.6s
[CV 3/5] END bootstrap=True, max_depth=80, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=400;, score=-0.185 total time=   4.5s
[CV 4/5] END bootstrap=True, max_depth=80, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=400;, score=-0.134 total time=   4.5s
[CV 5/5] END bootstrap=True, max_depth=80, max_features=None, min_samples_leaf=4, min_samples_split=2, n_estimators=400;, score=-0.134 total time=   4.4s
[CV 1/5] END bootstrap=False, max_depth=60, max_features=log2, min_samples_leaf=2, min_samples_split=5, n_estimators=300;, score=-0.167 total time=   0.5s
[CV 2/5] END b

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(), n_iter=15,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 60, 70,
                                                      80, 90, 100],
                                        'max_features': [1.0, 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600]},
                   scoring=make_scorer(mean_squared_log_error, greater_is_better=False, squared=False),
                   verbose=5)

In [16]:
hyper_tuning.best_score_

-0.14737667350638187

Selección de los mejores parametros de nuestro modelo para su posterior entrenamiento

In [17]:
hyper_tuning.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

In [20]:
RF_hyper_tuning = RandomForestRegressor(
    n_estimators=100,
    min_samples_split=10,
    min_samples_leaf= 1,
    max_features='sqrt',
    max_depth=100,
    bootstrap= False
)

In [21]:
RF_hyper_tuning.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, max_depth=100, max_features='sqrt',
                      min_samples_split=10)

In [22]:
print('Training score for Random Forest Regressor Hyper Parametric Tuning is ',RF_hyper_tuning.score(X_train,y_train))
print('Testing score for Random Forest Regressor Hyper Parametric Tuning is ',RF_hyper_tuning.score(X_test,y_test))

Training score for Random Forest Regressor Hyper Parametric Tuning is  0.9760742235117711
Testing score for Random Forest Regressor Hyper Parametric Tuning is  0.8628875171091486


### Esta es la metrica solicitada en el repositrio

In [23]:
mean_squared_log_error(y_test, RF_hyper_tuning.predict(X_test), squared=False)

0.1641342006681297

### CROSS VALIDATION -- RANDOM FOREST REGRESSOR

Se realizará un proceso de validación cruzada para evaluar el modelo con sobre diferentes formas de agrupar nuestros datos

In [24]:
RF_score = cross_val_score(
    RF_hyper_tuning, 
    X, 
    y, 
    cv=KFold(n_splits=5,
             shuffle=True, 
             random_state=42), 
    scoring=RMSLE
)

El valor final que se obtiene como promedio de la validacion cruzada realizada para nuestro dataset es 0.1496

In [25]:
print('RMSLE: ',np.abs(RF_score).mean())

RMSLE:  0.14961903027400422


## XTREME GRADIENT BOOSTING

In [26]:
XGBReg = xgb.XGBRegressor()

In [ ]:
XGBReg.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

In [28]:
mean_squared_log_error(y_test, XGBReg.predict(X_test), squared=False)

0.14633244025073427

## Hyper parametric Tuning -- Streme Gradient Boosting

In [29]:
params = {
    "colsample_bytree": np.linspace(0.1, 0.7, 10),
    "gamma": np.linspace(0, 0.5, 5),
    "learning_rate": np.linspace(0.1, 0.01, 10), # default 0.1 
    "max_depth": [1,2,4,6], # default 3
    "n_estimators": [i for i in range(500, 2100, 200)] # default 100
}

In [30]:
XGB_hyper_tuning = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(),
    param_distributions=params,
    random_state=42,
    n_iter=25,
    cv=KFold(n_splits=5,
            shuffle=True,
            random_state=42),
    n_jobs=-1,
    scoring=RMSLE
)

In [32]:
XGB_hyper_tuning.fit(X_train, y_train, eval_set=[(X_test, y_test)])

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[803]	validation_0-rmse:29147.9
[804]	validation_0-rmse:29148.6
[805]	validation_0-rmse:29147
[806]	validation_0-rmse:29147.2
[807]	validation_0-rmse:29146.7
[808]	validation_0-rmse:29146.4
[809]	validation_0-rmse:29146.1
[810]	validation_0-rmse:29146.2
[811]	validation_0-rmse:29144.8
[812]	validation_0-rmse:29145.4
[813]	validation_0-rmse:29145.1
[814]	validation_0-rmse:29146.1
[815]	validation_0-rmse:29145.9
[816]	validation_0-rmse:29145.8
[817]	validation_0-rmse:29145.9
[818]	validation_0-rmse:29145.6
[819]	validation_0-rmse:29145.2
[820]	validation_0-rmse:29145.3
[821]	validation_0-rmse:29144.6
[822]	validation_0-rmse:29144.4
[823]	validation_0-rmse:29144.1
[824]	validation_0-rmse:29144.2
[825]	validation_0-rmse:29144.1
[826]	validation_0-rmse:29143.8
[827]	validation_0-rmse:29143.7
[828]	validation_0-rmse:29143
[829]	validation_0-rmse:29142.3
[830]	validation_0-rmse:29141.5
[831]	validation_0-rmse:29140.6
[832]	val

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBRegressor(), n_iter=200, n_jobs=1,
                   param_distributions={'colsample_bytree': array([0.1       , 0.16666667, 0.23333333, 0.3       , 0.36666667,
       0.43333333, 0.5       , 0.56666667, 0.63333333, 0.7       ]),
                                        'gamma': array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444...
                                        'learning_rate': array([0.1       , 0.09526316, 0.09052632, 0.08578947, 0.08105263,
       0.07631579, 0.07157895, 0.06684211, 0.06210526, 0.05736842,
       0.05263158, 0.04789474, 0.04315789, 0.03842105, 0.03368421,
       0.02894737, 0.02421053, 0.01947368, 0.01473684, 0.01      ]),
                                        'max_depth': [1, 2, 4, 6],
                                        'n_estimators': [500, 700, 900, 1100,
                    

In [33]:
XGB_hyper_tuning.best_score_

-0.12085453993821463

In [44]:
XGB_hyper_tuning.best_params_

{'colsample_bytree': 0.23333333333333334,
 'gamma': 0.05555555555555555,
 'learning_rate': 0.01473684210526316,
 'max_depth': 4,
 'n_estimators': 1900}

In [45]:
XGBReg_hyper_tuning = xgb.XGBRegressor(
  colsample_bytree=0.23,
  gamma= 0.055,
  learning_rate= 0.0147,
  max_depth= 4,
  n_estimators= 1900
)

In [46]:
XGBReg_hyper_tuning.fit(X_train, y_train)

[02:47:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(colsample_bytree=0.23, gamma=0.055, learning_rate=0.0147,
             max_depth=4, n_estimators=1900)

In [47]:
print('Training score for Random Forest Regressor Hyper Parametric Tuning is ',XGBReg_hyper_tuning.score(X_train,y_train))
print('Testing score for Random Forest Regressor Hyper Parametric Tuning is ',XGBReg_hyper_tuning.score(X_test,y_test))

Training score for Random Forest Regressor Hyper Parametric Tuning is  0.992290486986679
Testing score for Random Forest Regressor Hyper Parametric Tuning is  0.9070187902886179


In [52]:
print('RMSLE: {:.4f}'.format(mean_squared_log_error(y_test, XGBReg_hyper_tuning.predict(X_test), squared=False)))

RMSLE: 0.1312


### CROSS VALIDATION -- XTREME GRADIENT BOOSTING

In [53]:
XGB_score = cross_val_score(
    XGBReg_hyper_tuning, 
    X, 
    y, 
    cv=KFold(n_splits=5,
             shuffle=True, 
             random_state=42), 
    scoring=RMSLE
)

[02:54:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:54:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:55:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Se realizará un proceso de validación cruzada para evaluar el modelo sobre diferentes formas de agrupar nuestros datos

In [56]:
print('RMSLE: {:.4f}'.format(np.abs(XGB_score).mean()))

RMSLE: 0.1247


El valor final que se obtiene como promedio de la validacion cruzada realizada para nuestro dataset es 0.1247

## CONCLUSIÓN
Se entrenó dos modelos, `RandomForestRegressor` y `XGBRegressor`, de estos se obtuvieron valores de métricas 0.1496 y 0.1247 respectivamente evaluandolos con RMSLE, por lo tanto el modelo que mejor generaliza y tiene mejor performace es `XGBRegressor`. Los siguientes resultados son predichos en base a este modelo.

## SAVING THE MODEL

Guardo el modelo para que proximamente se pueda usar sin necesidad de volver a entrenarlo

In [57]:
from joblib import dump, load
dump(XGBReg_hyper_tuning, 'XGBReg_hyper_tuning.joblib') 

['XGBReg_hyper_tuning.joblib']

In [58]:
model = load('XGBReg_hyper_tuning.joblib')

[03:10:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


## SAVING RESULTS IN A CSV

Por ultimo genero y almaceno las predicciones de mi modelo sobre los datos de prueba proporcionados en el repositorio

In [59]:
pred_test = model.predict(data_test)
pred_test

array([148998.62, 127479.09, 160623.16, ..., 164760.52, 120591.53,
       212960.89], dtype=float32)

In [67]:
pred_test_df = pd.DataFrame(
    {
        "pred": pred_test
    }
)
pred_test_df

,pred
0,148998.625000
1,127479.093750
2,160623.156250
3,183895.281250
4,194854.656250
...,...
1455,81791.796875
1456,74510.835938
1457,164760.515625
1458,120591.531250


In [69]:
pred_test_df.to_csv("pred_test.csv", index=False)